# Двухуровневые модели рекомендаций


Код для src, utils, metrics вы можете скачать из [этого](https://github.com/geangohn/recsys-tutorial) github репозитория

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit import als

# Модель второго уровня
from lightgbm import LGBMClassifier

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

# Написанные нами функции
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender

In [ ]:
data = pd.read_csv('retail_train.csv')
item_features = pd.read_csv('product.csv')
user_features = pd.read_csv('hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)


# Важна схема обучения и валидации!
# -- давние покупки -- | -- 6 недель -- | -- 3 недель -- 
# подобрать размер 2-ого датасета (6 недель) --> learning curve (зависимость метрики recall@k от размера датасета)
val_lvl_1_size_weeks = 6
val_lvl_2_size_weeks = 3

data_train_lvl_1 = data[data['week_no'] < data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)]
data_val_lvl_1 = data[(data['week_no'] >= data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)) &
                      (data['week_no'] < data['week_no'].max() - (val_lvl_2_size_weeks))]

data_train_lvl_2 = data_val_lvl_1.copy()  # Для наглядности. Далее мы добавим изменения, и они будут отличаться
data_val_lvl_2 = data[data['week_no'] >= data['week_no'].max() - val_lvl_2_size_weeks]

data_train_lvl_1.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [ ]:
n_items_before = data_train_lvl_1['item_id'].nunique()

data_train_lvl_1 = prefilter_items(data_train_lvl_1, item_features)

n_items_after = data_train_lvl_1['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

c:\python37\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Decreased # items from 83685 to 9499


In [ ]:
recommender = MainRecommender(data_train_lvl_1)

In [ ]:
recommender.get_als_recommendations(2375, N=5)

[981760, 899624, 844179, 1082185, 845208]

In [ ]:
# recommender.get_own_recommendations(2375, N=5)

In [ ]:
recommender.get_similar_items_recommendation(2375, N=5)

[971610, 1036501, 1068517, 878398, 918046]

In [ ]:
recommender.get_similar_users_recommendation(2375, N=5)

[971610, 1036501, 1090330, 878398, 927681]

### Задание 1

A) Попробуйте различные варианты генерации кандидатов. Какие из них дают наибольший recall@k ?
- Пока пробуем отобрать 50 кандидатов (k=50)
- Качество измеряем на data_val_lvl_1: следующие 6 недель после трейна

Дают ли own recommendtions + top-popular лучший recall?  

B)* Как зависит recall@k от k? Постройте для одной схемы генерации кандидатов эту зависимость для k = {20, 50, 100, 200, 500}  
C)* Исходя из прошлого вопроса, как вы думаете, какое значение k является наиболее разумным?


In [ ]:
result_lvl_1 = data_val_lvl_1.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_1.columns=['user_id', 'actual']
result_lvl_1.head(2)

,user_id,actual
0,1,"[853529, 865456, 867607, 872137, 874905, 87524..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870..."


In [ ]:
result_lvl_1['own'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=20))
result_lvl_1.head(2)

,user_id,actual,own
0,1,"[853529, 865456, 867607, 872137, 874905, 87524...","[856942, 1049998, 9297615, 1074612, 991580, 55..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870...","[1076580, 911974, 7433029, 5567582, 1070803, 8..."


In [ ]:
result_lvl_1.apply(lambda row: recall_at_k(row['own'], row['actual']), axis=1).mean()

0.023903602684069804

In [ ]:
result_lvl_1.apply(lambda row: precision_at_k(row['own'], row['actual']), axis=1).mean()

0.2251624883936862

In [ ]:
def popularity_recommendation(data, n):
    """Топ-n популярных товаров"""
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    recs = popular.head(n).item_id
    
    return recs.tolist()


popular_recs = popularity_recommendation(data_val_lvl_1, n=50)
result_lvl_1['popular_recommendation'] = result_lvl_1['user_id'].apply(lambda x: popular_recs)
result_lvl_1.apply(lambda row: recall_at_k(row['popular_recommendation'], row['actual']), axis=1).mean()

0.01587922675792705

### Задание 2.

Обучите модель 2-ого уровня, при этом:
    - Добавьте минимум по 2 фичи для юзера, товара и пары юзер-товар
    - Измерьте отдельно precision@5 модели 1-ого уровня и двухуровневой модели на data_val_lvl_2
    - Вырос ли precision@5 при использовании двухуровневой модели?

In [ ]:
users_lvl_2 = pd.DataFrame(data_train_lvl_2['user_id'].unique())
users_lvl_2.columns = ['user_id']

# Пока только warm start
train_users = data_train_lvl_1['user_id'].unique()
users_lvl_2 = users_lvl_2[users_lvl_2['user_id'].isin(train_users)]

#создание списков кандидатов на певом уровне
users_lvl_2['candidates'] = users_lvl_2['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=20))

#разворачивание списков в столбцы
df = pd.DataFrame({'user_id':users_lvl_2.user_id.values.repeat(len(users_lvl_2.candidates[0])),
                 'item_id':np.concatenate(users_lvl_2.candidates.values).astype('int64')})

In [ ]:
data_train_lvl_2.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
2104867,2070,40618492260,594,1019940,1,1.00,311,-0.29,40,86,0.0,0.0
2107468,2021,40618753059,594,840361,1,0.99,443,0.00,101,86,0.0,0.0


In [ ]:
# data_train_lvl_1
# data_val_lvl_1 == data_train_lvl_2
# data_val_lvl_2

#создание таргета на основе полученных на первом уровне кандидатов и совпадения их с валидационным датасетом для первого уровня
targets_lvl_2 = data_train_lvl_2.drop_duplicates(['user_id', 'item_id'])[['user_id', 'item_id']].copy()
targets_lvl_2['target'] = 1  # тут только покупки 
targets_lvl_2 = df.merge(targets_lvl_2, on=['user_id', 'item_id'], how='left')
targets_lvl_2['target'].fillna(0, inplace= True)
targets_lvl_2['target'].mean()

0.14760798885276358

In [ ]:
targets_lvl_2 = targets_lvl_2.merge(item_features, on='item_id', how='left')
targets_lvl_2 = targets_lvl_2.merge(user_features, on='user_id', how='left')
X_train = targets_lvl_2.drop(['user_id', 'item_id', 'target'], axis=1)
y_train = targets_lvl_2[['target']]
cat_feats = X_train.columns[1:].tolist()
X_train[cat_feats] = X_train[cat_feats].astype('category')

In [ ]:
X_train.head(2)

,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
0,1208.0,GROCERY,National,ISOTONIC DRINKS,ISOTONIC DRINKS SINGLE SERVE,32 OZ,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown
1,2224.0,GROCERY,National,SOFT DRINKS,SFT DRNK SNGL SRV BTL CARB (EX,20 OZ,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown


In [ ]:
lgb = LGBMClassifier(objective='binary', max_depth=7, categorical_column=cat_feats)
lgb.fit(X_train, y_train)

train_preds = lgb.predict_proba(X_train)[:, 1]
train_preds

c:\python37\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
c:\python37\lib\site-packages\lightgbm\basic.py:863: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


array([0.16576526, 0.30294876, 0.18962521, ..., 0.35852825, 0.31058005,
       0.38458356])

In [ ]:
result_lvl_2 = data_val_lvl_2.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_2.columns=['user_id', 'actual']

df_res = targets_lvl_2[['user_id', 'item_id']]

#добавление неотсортированных кандидатов
candidats = df_res.groupby('user_id')['item_id'].apply(list)
result_lvl_2 = result_lvl_2.merge(candidats, on=['user_id'], how='left')
result_lvl_2 = result_lvl_2.rename(columns={'item_id': 'own'})

#добавление отсортированных по LGBMClassifier кандидатов
df_res['train_preds'] = train_preds
df_res.sort_values('train_preds', ascending=False, inplace=True)
top_candidats = df_res.groupby('user_id')['item_id'].apply(list)
result_lvl_2 = result_lvl_2.merge(top_candidats, on=['user_id'], how='left')
result_lvl_2 = result_lvl_2.rename(columns={'item_id': 'lgb'})

c:\python37\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
c:\python37\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [ ]:
from sklearn.metrics import precision_recall_curve, roc_curve, roc_auc_score, confusion_matrix, roc_auc_score

precision, recall, thresholds = precision_recall_curve(y_train, df_res['train_preds'])

fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))

roc_auc = roc_auc_score(y_true=y_train, y_score=df_res['train_preds'])
print(roc_auc)

Best Threshold=0.876701, F-Score=nan, Precision=0.000, Recall=0.000
0.6318872849343488


c:\python37\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in true_divide
  """


In [ ]:
#таргеты юзера 1
targets_1 = targets_lvl_2[['user_id', 'item_id', 'target']].loc[targets_lvl_2['user_id'] == 1]
#предсказание LGBMClassifier для юзера 1
preds_1 = df_res.loc[df_res['user_id'] == 1]

preds_1.merge(targets_1, on='item_id', how='left')

,user_id_x,item_id,train_preds,user_id_y,target
0,1,9655212,0.567476,1,1.0
1,1,1082269,0.551443,1,1.0
2,1,854920,0.421260,1,0.0
3,1,8293439,0.407031,1,1.0
4,1,931136,0.398177,1,1.0
5,1,1006546,0.377949,1,1.0
6,1,991580,0.361299,1,1.0
7,1,8090541,0.306996,1,0.0
8,1,1074612,0.305272,1,1.0
9,1,856942,0.302583,1,1.0


In [ ]:
result_lvl_2

,user_id,actual,own,lgb
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[856942, 1049998, 9297615, 1074612, 991580, 55...","[9655212, 1082269, 854920, 8293439, 931136, 10..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...",NaN,NaN
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[13003092, 995598, 5570685, 5569792, 1108624, ...","[6979393, 5580166, 9831557, 996269, 1119051, 1..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[845814, 1075524, 1097544, 1112957, 949836, 99...","[840386, 9338009, 1021133, 945627, 845814, 875..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[8181377, 12808385, 955879, 981660, 1116578, 9...","[1116578, 1021715, 8181377, 5577022, 911703, 9..."
...,...,...,...,...
2037,2496,[6534178],"[10285187, 900770, 10285149, 1076744, 992870, ...","[855672, 887783, 992870, 842783, 5573314, 8655..."
2038,2497,"[1016709, 9835695, 1132298, 16809501, 845294, ...","[1066685, 1033913, 1102207, 896938, 1031864, 8...","[896938, 1102207, 838487, 1135834, 820361, 106..."
2039,2498,"[15716530, 834484, 901776, 914190, 958382, 972...","[1022066, 1076580, 5565356, 931579, 1100379, 9...","[851101, 968759, 957891, 1100379, 5577022, 107..."
2040,2499,"[867188, 877580, 902396, 914190, 951590, 95813...","[1128395, 5568964, 1069256, 6904613, 889989, 5...","[866528, 6904613, 866292, 833458, 873964, 5570..."


In [ ]:
result_lvl_2 = result_lvl_2.loc[~result_lvl_2['lgb'].isnull()]

In [ ]:
result_lvl_2.apply(lambda row: recall_at_k(row['lgb'], row['actual']), axis=1).mean()

0.021493774576613094

In [ ]:
result_lvl_2.apply(lambda row: precision_at_k(row['lgb'], row['actual']), axis=1).mean()

0.19394255874673627

In [ ]:
result_lvl_2.apply(lambda row: recall_at_k(row['own'], row['actual']), axis=1).mean()

0.023124002872337242

In [ ]:
result_lvl_2.apply(lambda row: precision_at_k(row['own'], row['actual']), axis=1).mean()

0.18924281984334201

### Финальный проект

Мы уже прошли всю необходимуб теорию для финального проекта. Проект осуществляется на данных из вебинара (данные считаны в начале ДЗ).
Рекомендуем вам **начать делать проект сразу после этого домашнего задания**
- Целевая метрика - precision@5. Порог для уcпешной сдачи проекта precision@5 > 0.27%
- Будет public тестовый датасет, на котором вы сможете измерять метрику
- Также будет private тестовый датасет для измерения финального качества
- НЕ обязательно, но крайне желательно использовать 2-ух уровневые рекоммендательные системы в проекте
- Вы сдаете код проекта в виде github репозитория и csv файл с рекомендациями 